In [13]:
pip install image_tabular

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [2]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWg9Ne0LrmvmmQFnPH-1Ok9RmVn-qh-KdLUr1hrvRIWNqwz66YwTJ7s
Mounted at /content/drive


In [3]:
import os
#set working directory
root_dir = "/content/drive/My Drive/"

In [4]:
#choose the project folder
project_folder = "Colab Notebooks/chest/"

In [5]:
#define a function to create and set the working directory
def create_and_set_working_directory(project_folder):
  #check if the project folder exists. if not, make one.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)

print(root_dir + project_folder + 'did not exist and was created.')

/content/drive/My Drive/Colab Notebooks/chest/did not exist and was created.


In [6]:
#change the OS path to project folder as working directory
os.chdir(root_dir + project_folder)

In [7]:
#create a test file in the working directory and see if it shows up at the right place
!touch 'new_file_test.txt'
print('working directory' + root_dir + project_folder + \
      "empty text file created. You can also run !pwd command to confirm working directory.")

working directory/content/drive/My Drive/Colab Notebooks/chest/empty text file created. You can also run !pwd command to confirm working directory.


In [8]:
pwd

'/content/drive/My Drive/Colab Notebooks/chest'

In [11]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [14]:
from fastai.vision import *
from image_tabular.core import *
from image_tabular.metric import *

# use gpu by default if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

In [17]:

data_path = Path("data/train/")

In [21]:
#CSV data 
train_df = pd.read_csv("ann_pneumonia.csv")
# test_df = pd.read_csv(data_path/"test.csv")

# print(len(train_df), len(test_df))
print(len(train_df))

912


In [22]:
train_df.head()

,Image Index,Follow-up,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,Height,OriginalImagePixelSpacingx,y,label
0,00000002_000.png,0,2,80,M,PA,2500,2048,0.171000,0.171000,0
1,00000005_000.png,0,5,69,F,PA,2048,2500,0.168000,0.168000,0
2,00000006_000.png,0,6,81,M,PA,2500,2048,0.168000,0.168000,0
3,00000007_000.png,0,7,82,M,PA,2500,2048,0.168000,0.168000,0
4,00000013_000.png,25,13,60,M,PA,2021,2021,0.194311,0.194311,0


In [24]:
# extremely unbalanced dataset, most of the images are benign
train_df["label"].value_counts(normalize=True)

0    0.767544
1    0.232456
Name: label, dtype: float64

In [ ]:
# some columns are totally unproductive so let's remove them
data.drop([ 'Patient ID','OriginalImageWidth','Height','Follow-up', 'Patient Age', 'Patient Gender', 'View Position',
       'OriginalImagePixelSpacingx', 'y', ],axis=1,inplace=True)

In [ ]:
dummies.head()

,label
Image Index,
00000002_000.png,0
00000005_000.png,0
00000006_000.png,0
00000007_000.png,0
00000013_000.png,0


In [ ]:
#Datasets as numpy arrays
train_data = np.array(dummies, dtype = 'float32')

In [ ]:
#Normalize / scale X values
X_train =train_data[:, 1:] /255.
# X_test = train_data[:, 1:] /255

In [ ]:
#Convert y to categorical if planning on using categorical cross entropy
#No need to do this if using sparse categorical cross entropy
y_train = train_data[:, 0]
y_train_cat = to_categorical(y_train, num_classes=25)

# Image data

In [25]:
tfms = get_transforms(flip_vert=True)
size = 128

In [26]:
# idx for validation, shared by image and tabular data
val_idx = get_valid_index(train_df)
len(val_idx)

182

In [32]:
# load image data using train_df and prepare fastai LabelLists
image_data = (ImageList.from_df(train_df, path=data_path, cols="Image Index",
                               folder="normal")
              .split_by_idx(val_idx)
              .label_from_df(cols="label")
              .transform(tfms, size=size))
#, suffix=".png"

/usr/local/lib/python3.7/dist-packages/fastai/vision/transform.py:247: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:766.)
  return _solve_func(B,A)[0][:,0]


In [34]:
# add test data so that we can make predictions
test_image_data = ImageList.from_df(test_df, path=data_path, cols="Image Index",
                                    folder="pneumonia")
image_data.add_test(test_image_data)
#, suffix=".jpg"

NameError: ignored

In [ ]:
#Apply CNN
# ### Build the model

#############################################################
###2 conv and pool layers. with some normalization and drops in between.

INPUT_SHAPE = (SIZE, SIZE, 1)   #change to (SIZE, SIZE, 3)
inp = keras.layers.Input(shape=INPUT_SHAPE)

conv1 = keras.layers.Conv2D(32, kernel_size=(3, 3), 
                               activation='relu', padding='same')(inp)
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
norm1 = keras.layers.BatchNormalization(axis = -1)(pool1)
drop1 = keras.layers.Dropout(rate=0.2)(norm1)
conv2 = keras.layers.Conv2D(32, kernel_size=(3, 3), 
                               activation='relu', padding='same')(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
norm2 = keras.layers.BatchNormalization(axis = -1)(pool2)
drop2 = keras.layers.Dropout(rate=0.2)(norm2)

flat = keras.layers.Flatten()(drop2)  #Flatten the matrix to get it ready for dense.

hidden1 = keras.layers.Dense(512, activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis = -1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.2)(norm3)
hidden2 = keras.layers.Dense(256, activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis = -1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.2)(norm4)

out = keras.layers.Dense(2, activation='sigmoid')(drop4)   #units=1 gives error

model = keras.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam',
                loss='categorical_crossentropy',   #Check between binary_crossentropy and categorical_crossentropy
                metrics=['accuracy'])
print(model.summary())
    

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 256, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 128, 128, 32)     128       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 128, 128, 32)      0         
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 32)      9248

In [ ]:
 ### Split the dataset
# 
# I split the dataset into training and testing dataset.
# 1. Training data: 80%
# 2. Testing data: 20%
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(dataset, to_categorical(np.array(label)), test_size = 0.20, random_state = 0)


In [ ]:

#Before fitting the model the training data write the below 2 lines:

X_train=np.asarray(X_train).astype(np.int)

y_train=np.asarray(y_train).astype(np.int)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


TypeError: ignored

In [ ]:
#Fit the model
history = model.fit(np.array(X_train), y_train, batch_size = 64, verbose = 1, epochs = 5, validation_split = 0.1,shuffle = False)     #Changed to 3 from 50 for testing purposes.
                         
                      #   callbacks=callbacks
                     


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


ValueError: ignored